In [8]:
import urllib.request
from urllib.request import Request, urlopen

import requests
import json
import jsonpath

import jsonlines

import pandas as pd

'''
# request url

#url = "https://api-samenmeten.rivm.nl/v1.0/Things"
#response = requests.get(url)
#html = response.text

response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()

# creat columns
data = json.loads(raw_data)
a = "name"
station_name = jsonpath.jsonpath(data, "$..name")
station_id = jsonpath.jsonpath(data,"@iot.selfLink")

print (station_name)
#station_coordinates = jsonpath.jsonpath(data,'$..coordinates')
'''

# request url
response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()
str_data = bytes.decode(raw_data)
json_data = json.loads(str_data)

print(type(json_data))
print(json_data.keys())

station_id = pd.json_normalize(json_data, record_path=['value'])[['@iot.id']]
station_name = pd.json_normalize(json_data, record_path=['value'])[['name']]
station_owner = pd.json_normalize(json_data,  record_path=['value'])[['properties.owner']]
station_Datastreamslink = pd.json_normalize(json_data,  record_path=['value'])[['Datastreams@iot.navigationLink']]
print (station_id)
print (station_name)
print (station_owner)
print (station_Datastreamslink)

a = 0
b = 0
location_link = pd.json_normalize(json_data, record_path = ['value'])[['Locations@iot.navigationLink']]
station_lat2 = pd.DataFrame(columns = ['latitude'])
station_lon2 = pd.DataFrame(columns = ['longitude'])

while a < len(location_link):
    #step1：取出某行的url
    url_location = str(location_link.loc[a,'Locations@iot.navigationLink'])

    #step2：根据这个url读取出数据，转变成pandas的数据结构
    response_location = urlopen (url_location)
    raw_location = response_location.read()
    str_location = bytes.decode(raw_location)
    json_location = json.loads(str_location)

    #step3：取出这个数据中的具体值
    station_coordinates = pd.json_normalize(json_location, record_path=['value','location','coordinates'])
    station_lat = station_coordinates.loc[0,0]
    station_lon = station_coordinates.loc[1,0]

    #step4：把这个值放入另一列的某一行
    station_lat2.loc[b,'latitude'] = station_lat
    station_lon2.loc[b,'longitude'] = station_lon

    a = a + 1
    b = b + 1

    #step5：循环这个过程
print (station_lat2)
print (station_lon2)

'''
1.取出某行的url
2.根据这个url读取出数据，转变成pandas的数据结构
3.取出这个数据中的具体值
4.把这个值放入另一列的某一行
5.循环这个过程
'''



Joint1 = pd.concat([station_id, station_name, station_owner, station_lat2, station_lon2, station_Datastreamslink], axis=1)
Joint1.rename(columns = {'Datastreams@iot.navigationLink' : '@iot.selfLink'}, inplace = True)
Joint1.set_index('@iot.selfLink',inplace = True)
print (Joint1)

a = 0
b = 0
station_ave_result = pd.DataFrame(columns = ['average_value'])
Joint2 = pd.DataFrame(columns = ['@iot.selfLink', 'description', 'average_value', 'unitOfMeasurement.symbol'])
datastreams_link = pd.json_normalize(json_data, record_path = ['value'])[['Datastreams@iot.navigationLink']]

while a < len (datastreams_link):
    url_datastreams = str(datastreams_link.loc[a,'Datastreams@iot.navigationLink'])
    response_datastreams = urlopen (url_datastreams)
    raw_datastreams = response_datastreams.read()
    str_datastreams = bytes.decode(raw_datastreams)
    json_datastreams = json.loads(str_datastreams)
    
    station_symbols = pd.json_normalize(json_datastreams,  record_path = ['value'])[['unitOfMeasurement.symbol']]
    station_descriptions = pd.json_normalize(json_datastreams,  record_path = ['value'])[['description']]
    station_datastream_selflinks = pd.json_normalize(json_datastreams,  record_path = ['value'])[['@iot.selfLink']]
    station_datastream_selflinks.iloc[:,0] = url_datastreams
    
#    print (datastreams_link)
#    print (station_descriptions)
#    print (station_datastream_selflinks)
    
    
    obeservations_link = pd.json_normalize(json_datastreams, record_path = ['value'])[['Observations@iot.navigationLink']]
    
    while b < len (obeservations_link):
        url_obeservations = str(obeservations_link.loc[b,'Observations@iot.navigationLink'])
        response_obeservations = urlopen (url_obeservations)
        raw_obeservations = response_obeservations.read()
        str_obeservations = bytes.decode(raw_obeservations)
        json_obeservations = json.loads(str_obeservations)
        
        try:
            station_resulttime = pd.json_normalize(json_obeservations,  record_path = ['value'])[['phenomenonTime']]
        except:
            station_ave_result.loc[b,'average_value'] =  None
        else:
            station_results = pd.json_normalize(json_obeservations,  record_path = ['value'])[['result']]
            station_ave_result.loc[b,'average_value'] = station_results['result'].mean()
            
        b = b + 1
        
    Combination = pd.concat([station_datastream_selflinks, station_descriptions, station_ave_result, station_symbols], axis=1)
    Joint2 = pd.concat([Joint2, Combination], ignore_index = True)
    
    a = a + 1
    b = 0
    station_ave_result = pd.DataFrame(columns = ['average_value'])


print (len (obeservations_link))
print (station_resulttime)
print (Joint2)

'''
1.先把单个thing（starion）的重要列（需要记录的测量值）算出来（由darastreamslink标识）
2.创造一个新列，命名为平均值
3.平均值的计算
A.先把其中一个记录类型通过obsevationlink连接的json数据提取出来
B.把result做成dataframe的列
C.计算该列的平均值
D.循环
2.把这些重要列合成一个组,把这个组放到Joint2下面
3.循环
'''

result = Joint2.join(Joint1, on='@iot.selfLink')
print (result)
result.to_excel('open_air_pollution.xlsx')

                                          @iot.selfLink  \
0     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
1     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
2     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
3     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
4     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
...                                                 ...   
1066  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1067  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1068  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1069  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1070  https://api-samenmeten.rivm.nl/v1.0/Things(633...   

                          description average_value unitOfMeasurement.symbol  \
0          LUC_354724647885109-4-pres    999.272857                      hPa   
1            LUC_354724647885109-4-rh     40.230714                        %   
2          LUC_354724647885109-4-temp     16.122857